# CitiBike Data Cleaning: December 2023

The purpose of this notebook is to check a sample of the data we will be working with. In particular, we check for what needs to be cleaned, so that we can use the methods recursively for data from January 2023 to October 2025.

## 1. Imports

In [165]:
import numpy as np
import pandas as pd

df_1 = pd.read_csv("data/2023-citibike-tripdata/202312-citibike-tripdata/202312-citibike-tripdata_1.csv", low_memory=False)
df_2 = pd.read_csv("data/2023-citibike-tripdata/202312-citibike-tripdata/202312-citibike-tripdata_2.csv", low_memory=False)
df_3 = pd.read_csv("data/2023-citibike-tripdata/202312-citibike-tripdata/202312-citibike-tripdata_3.csv", low_memory=False)
df = pd.concat([df_1, df_2, df_3], ignore_index=True)

## 2. Overview

In [166]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2204874 entries, 0 to 2204873
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 218.7+ MB
None


In [167]:
df.isnull().sum()

ride_id                  0
rideable_type            0
started_at               0
ended_at                 0
start_station_name    1550
start_station_id      1550
end_station_name      6471
end_station_id        6471
start_lat             1550
start_lng             1550
end_lat               6452
end_lng               6452
member_casual            0
dtype: int64

In [117]:
df["start_station_id"] = pd.to_numeric(df["start_station_id"], errors="coerce")
df["end_station_id"]   = pd.to_numeric(df["end_station_id"],   errors="coerce")

start_id_name_counts = (
    df.groupby("start_station_id")["start_station_name"]
      .nunique()
      .sort_values(ascending=False)
)

start_id_name_counts.head(20)


start_station_id
3919.07    2
2733.03    1
6839.04    1
6869.06    1
6865.03    1
6860.12    1
6857.09    1
6855.02    1
6854.05    1
6850.01    1
6848.04    1
6847.05    1
6847.02    1
6842.02    1
6840.02    1
6839.10    1
6836.07    1
6801.01    1
6830.02    1
6827.11    1
Name: start_station_name, dtype: int64

In [118]:
end_id_name_counts = (
    df.groupby("end_station_id")["end_station_name"]
      .nunique()
      .sort_values(ascending=False)
)

end_id_name_counts.head(20)


end_station_id
3919.07    2
2733.03    1
6839.04    1
6869.06    1
6865.03    1
6860.12    1
6857.09    1
6855.02    1
6854.05    1
6850.01    1
6848.04    1
6847.05    1
6847.02    1
6842.02    1
6840.02    1
6839.10    1
6836.07    1
6801.01    1
6830.02    1
6827.11    1
Name: end_station_name, dtype: int64

In [124]:
start_name_id_counts = (
    df.groupby("start_station_name")["start_station_id"]
      .nunique()
      .sort_values(ascending=False)
)

start_name_id_counts.head(20)


start_station_name
1 Ave & E 110 St                   1
MacDonough St & Marcy Ave          1
Louis Nine Blvd & Intervale Ave    1
Lott St & Erasmus St               1
Loring Pl North & W 183 St         1
Lorimer St & Broadway              1
Longwood Ave & Southern Blvd       1
Longfellow Ave & E 165 St          1
Longfellow Ave & Aldus St          1
Little West St & 1 Pl              1
Lispenard St & Broadway            1
Linden St & Knickerbocker Ave      1
Linden Blvd & Nostrand Ave         1
Lincoln Rd & Ocean Ave             1
Lincoln Pl & Nostrand Ave          1
Lincoln Pl & Classon Ave           1
Lincoln Ave & E 138 St             1
Lincoln Ave & E 134 St             1
Liberty St & Broadway              1
Lexington Ave & Stuyvesant Ave     1
Name: start_station_id, dtype: int64

In [120]:
end_name_id_counts = (
    df.groupby("end_station_name")["end_station_id"]
      .nunique()
      .sort_values(ascending=False)
)

In [58]:
print(df["start_station_name"].nunique())
print(df["end_station_name"].nunique())
print(df["start_station_id"].nunique())
print(df["end_station_id"].nunique())

2134
2161
4298
2190


In [63]:
df["start_stat_id"] = pd.to_numeric(df["start_station_id"], errors="coerce").astype("float32")

In [121]:
pair_counts = (
    df.groupby(["start_station_name", "start_station_id"])
      .size()
      .reset_index(name="n")
)

In [122]:
canonical_ids = (
    pair_counts.sort_values(["start_station_name", "n"], ascending=[True, False])
               .drop_duplicates(subset=["start_station_name"])
               .rename(columns={"start_station_id": "canonical_station_id"})
               [["start_station_name", "canonical_station_id"]]
)

In [123]:
df = df.merge(canonical_ids, on="start_station_name", how="left")
df["start_station_id"] = df["canonical_station_id"]
df = df.drop(columns=["canonical_station_id"])


## 3. Imposing data types

In [168]:
# ride IDs and names → strings
for col in ["ride_id", "start_station_name", "end_station_name"]:
    if col in df.columns:
        df[col] = df[col].astype("string")
        
# station ids → numeric
for col in ["start_station_id", "end_station_id"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Categorical fields (lower memory usage)
for col in ["rideable_type", "member_casual"]:
    if col in df.columns:
        df[col] = df[col].astype("category")

# Coordinates → float32 (lower memory usage)
for col in ["start_lat", "start_lng", "end_lat", "end_lng"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce").astype("float32")

# Datetimes
for col in ["started_at", "ended_at"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce", utc=True)


In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2204874 entries, 0 to 2204873
Data columns (total 13 columns):
 #   Column              Dtype              
---  ------              -----              
 0   ride_id             string             
 1   rideable_type       category           
 2   started_at          datetime64[ns, UTC]
 3   ended_at            datetime64[ns, UTC]
 4   start_station_name  string             
 5   start_station_id    float64            
 6   end_station_name    string             
 7   end_station_id      float64            
 8   start_lat           float32            
 9   start_lng           float32            
 10  end_lat             float32            
 11  end_lng             float32            
 12  member_casual       category           
dtypes: category(2), datetime64[ns, UTC](2), float32(4), float64(2), string(3)
memory usage: 155.6 MB


## 3. Removing `start_station_id` and `end_station_id`

**Explanation**:
The CitiBike tripdata contains two ID fields:
`start_station_id`, `end_station_id`
However, these fields are not reliable identifiers, for multiple reasons:
- IDs change over time even when the station stays the same (renumbering, relocations).
- They often contain malformed values such as " " or embedded quotes.
- Many IDs are incorrectly parsed as floats, leading to artificial duplicates (e.g., 7522.02 vs '7522.02').
- A single station name frequently maps to multiple IDs, even within the same month.

**In contrast:**
- Station names and
- Coordinates (lat/lng)

Therefore, start_station_id and end_station_id should be removed from the cleaned dataset.

In [170]:
start_id_name_counts = (
    df.groupby("start_station_id")["start_station_name"]
      .nunique()
      .sort_values(ascending=False)
)

start_id_name_counts.head(20)

start_station_id
3919.07    2
2733.03    1
6839.04    1
6869.06    1
6865.03    1
6860.12    1
6857.09    1
6855.02    1
6854.05    1
6850.01    1
6848.04    1
6847.05    1
6847.02    1
6842.02    1
6840.02    1
6839.10    1
6836.07    1
6801.01    1
6830.02    1
6827.11    1
Name: start_station_name, dtype: int64

In [171]:
end_id_name_counts = (
    df.groupby("end_station_id")["end_station_name"]
      .nunique()
      .sort_values(ascending=False)
)

end_id_name_counts.head(20)

end_station_id
3919.07    2
2733.03    1
6839.04    1
6869.06    1
6865.03    1
6860.12    1
6857.09    1
6855.02    1
6854.05    1
6850.01    1
6848.04    1
6847.05    1
6847.02    1
6842.02    1
6840.02    1
6839.10    1
6836.07    1
6801.01    1
6830.02    1
6827.11    1
Name: end_station_name, dtype: int64

✅ Code to generate a clean 1-to-1 mapping
Count how many times each (name, ID) pair appears:

In [172]:
start_map = (
    df.groupby(["start_station_id", "start_station_name"])
      .size()
      .reset_index(name="n")
      .sort_values(["start_station_id", "n"], ascending=[True, False])
      .drop_duplicates(subset=["start_station_id"])
      .set_index("start_station_id")["start_station_name"]
)

end_map = (
    df.groupby(["end_station_id", "end_station_name"])
      .size()
      .reset_index(name="n")
      .sort_values(["end_station_id", "n"], ascending=[True, False])
      .drop_duplicates(subset=["end_station_id"])
      .set_index("end_station_id")["end_station_name"]
)

df["start_station_name"] = df["start_station_id"].map(start_map)
df["end_station_name"]   = df["end_station_id"].map(end_map)

In [181]:
df.dropna(subset=["start_station_id", "end_station_id"], inplace=True)

In [173]:
start_id_name_counts = (
    df.groupby("start_station_id")["start_station_name"]
      .nunique()
      .sort_values(ascending=False)
)

start_id_name_counts.head(20)

start_station_id
2733.03    1
6876.04    1
6869.10    1
6869.06    1
6865.03    1
6860.12    1
6857.09    1
6855.02    1
6854.05    1
6850.01    1
6848.04    1
6847.05    1
6847.02    1
6842.02    1
6840.02    1
6839.10    1
6839.04    1
6836.07    1
6830.02    1
6827.11    1
Name: start_station_name, dtype: int64

In [174]:
end_id_name_counts = (
    df.groupby("end_station_id")["end_station_name"]
      .nunique()
      .sort_values(ascending=False)
)

end_id_name_counts.head(20)

end_station_id
2733.03    1
6873.01    1
6869.06    1
6865.03    1
6860.12    1
6857.09    1
6855.02    1
6854.05    1
6850.01    1
6848.04    1
6847.05    1
6847.02    1
6842.02    1
6840.02    1
6839.10    1
6839.04    1
6836.07    1
6830.02    1
6827.11    1
6825.03    1
Name: end_station_name, dtype: int64

In [182]:
df.isnull().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

In [ ]:
# OLD CODE BELOW FOR REFERENCE
import pandas as pd

# Demonstrate unreliability of station_id fields:
print("Unique start_station_id:", df["start_station_id"].nunique())
print("Unique start_station_name:", df["start_station_name"].nunique())

mapping = (
    df.groupby(["start_station_name", "start_station_id"])
      .size()
      .reset_index(name="count")
)

#print(mapping.head(10))

# Result: The same station name appears with multiple station IDs,
# and many IDs contain trailing quotes or malformed values.

df = df.drop(columns=["start_station_id", "end_station_id"])

Unique start_station_id: 2222
Unique start_station_name: 2134


## 4. Removing observations with missing `start_lat` and `start_lng`

**Explanation**:

If a ride is missing both start_lat and start_lng, then:
- There is no start location in any form (station name and id are also always missing in these rows)
- The observation is unusable for spatial analysis
- It cannot be mapped, assigned to a zip code, or used for risk modeling

No imputation is possible (no coordinates, no station name, no station ID)

In [176]:
missing_start = df[df["start_lat"].isna() & df["start_lng"].isna()]
print("Missing start coords:", len(missing_start))

# Drop these rows:
df = df.dropna(subset=["start_lat", "start_lng"])


Missing start coords: 1550


## 5. Removing observations with missing `end_lat` and `end_lng`

**Explanation**:

Similarly, if end_lat and end_lng are both missing:
- There is no end location (end_station_name and end_station_id are always missing here)
- The trip endpoint is completely unknown
- You cannot compute distance, assign crash exposure, or perform any geospatial analysis

Note: _These represent ~0.3% of trips in December 2023 — a negligible amount to drop._

In [177]:
missing_end = df[df["end_lat"].isna() & df["end_lng"].isna()]
print("Missing end coords:", len(missing_end))

# Drop these:
df = df.dropna(subset=["end_lat", "end_lng"])


Missing end coords: 5643


## 6. Handling remaining missing `*_station_name` values

**Eplanation**:

After removing corrupted rows, the only remaining missing station names are:
- end_station_name missing (but end coordinates exist)

Because:
- They have valid coordinates
- They contribute meaningfully to spatial exposure, distance estimation, endpoint clustering, etc.

→ It is incorrect to drop these observations.

Since these trips do not have a station, the correct label for station name is: `Unknown`

In [179]:
# Count remaining missing names
remaining_missing = df["end_station_name"].isna().sum()
print("Missing end_station_name (with coordinates present):", remaining_missing)

# Replace NaN with "Unknown"
df["start_station_name"] = df["start_station_name"].fillna("Unknown")
df["end_station_name"] = df["end_station_name"].fillna("Unknown")


Missing end_station_name (with coordinates present): 478


## 7. Final cleaned data

In [129]:
df.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
Index: 2197042 entries, 0 to 2204873
Data columns (total 13 columns):
 #   Column              Dtype              
---  ------              -----              
 0   ride_id             string             
 1   rideable_type       category           
 2   started_at          datetime64[ns, UTC]
 3   ended_at            datetime64[ns, UTC]
 4   start_station_name  string             
 5   start_station_id    float64            
 6   end_station_name    string             
 7   end_station_id      float64            
 8   start_lat           float32            
 9   start_lng           float32            
 10  end_lat             float32            
 11  end_lng             float32            
 12  member_casual       category           
dtypes: category(2), datetime64[ns, UTC](2), float32(4), float64(2), string(3)
memory usage: 171.8 MB


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5301271E68E18BA4,electric_bike,2023-12-09 14:36:16.912000+00:00,2023-12-09 14:48:13.919000+00:00,Watts St & Greenwich St,5578.02,W 4 St & 7 Ave S,5880.02,40.724056,-74.009659,40.734013,-74.002937,casual
1,DE4CC2FEB483AE3A,electric_bike,2023-12-13 09:50:39.182000+00:00,2023-12-13 10:13:01.348000+00:00,Underhill Ave & Lincoln Pl,4042.08,Plaza St East & Flatbush Ave,4010.01,40.674011,-73.967148,40.673134,-73.969109,casual
2,0C67D9174CBDE2D0,electric_bike,2023-12-02 13:00:29.094000+00:00,2023-12-02 13:33:22.654000+00:00,W Broadway & Spring St,5569.06,Central Park W & W 103 St,7577.27,40.724945,-74.001656,40.795589,-73.961884,casual
3,BDB5F8E57AF7CC70,classic_bike,2023-12-08 11:09:49.012000+00:00,2023-12-08 11:11:21.778000+00:00,Ave A & E 11 St,5703.13,E 11 St & 1 Ave,5746.14,40.728546,-73.981758,40.729538,-73.984268,member
4,599A6C20123BED9D,electric_bike,2023-12-06 10:12:58.286000+00:00,2023-12-06 10:19:12.479000+00:00,Ave A & E 11 St,5703.13,Lafayette St & Jersey St,5561.06,40.728546,-73.981758,40.724560,-73.995651,casual


In [130]:
df.isnull().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

In [48]:
df["start_station_name"].nunique(), df["end_station_name"].nunique()

(2134, 2161)

In [49]:
df["new_start_station_name"] = df["start_station_name"].str.replace(r"\s+", " ", regex=True).str.lower().str.strip()
df["new_end_station_name"] = df["end_station_name"].str.replace(r"\s+", " ", regex=True).str.lower().str.strip()
df["new_start_station_name"].nunique(), df["new_end_station_name"].nunique()

(2134, 2161)

In [65]:
import urllib.request, json 
import pandas as pd
# from pandas.io.json import json_normalize

with urllib.request.urlopen('https://gbfs.citibikenyc.com/gbfs/en/station_information.json') as url:
    data = json.loads(url.read().decode())

In [69]:
data["data"]["stations"][0]

{'station_type': 'classic',
 'rental_methods': ['KEY', 'CREDITCARD'],
 'region_id': '71',
 'capacity': 21,
 'name': 'E New York Ave & Troy Ave',
 'short_name': '3680.04',
 'lat': 40.66227,
 'electric_bike_surcharge_waiver': False,
 'eightd_station_services': [],
 'eightd_has_key_dispenser': False,
 'external_id': '2c8b263e-952c-4941-a671-ef0ebb062632',
 'rental_uris': {'ios': 'https://bkn.lft.to/lastmile_qr_scan',
  'android': 'https://bkn.lft.to/lastmile_qr_scan'},
 'lon': -73.93762,
 'has_kiosk': True,
 'station_id': '2c8b263e-952c-4941-a671-ef0ebb062632'}

In [73]:
data.keys()

dict_keys(['data', 'last_updated', 'ttl', 'version'])

In [77]:
stations = pd.DataFrame(data["data"]["stations"])

In [78]:
stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   station_type                    2310 non-null   object 
 1   rental_methods                  2310 non-null   object 
 2   region_id                       2300 non-null   object 
 3   capacity                        2310 non-null   int64  
 4   name                            2310 non-null   object 
 5   short_name                      2310 non-null   object 
 6   lat                             2310 non-null   float64
 7   electric_bike_surcharge_waiver  2310 non-null   bool   
 8   eightd_station_services         2310 non-null   object 
 9   eightd_has_key_dispenser        2310 non-null   bool   
 10  external_id                     2310 non-null   object 
 11  rental_uris                     2310 non-null   object 
 12  lon                             23

In [79]:
stations["short_name"].nunique()

2310

In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2204874 entries, 0 to 2204873
Data columns (total 14 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
 13  start_stat_id       float32
dtypes: float32(1), float64(4), object(9)
memory usage: 227.1+ MB


In [86]:
df["start_station_id"].nunique()

4298

In [93]:
mask = df["start_station_id"].astype("str").isin(stations["short_name"])

In [94]:
df[~mask]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
51,C110098C96C7AD22,classic_bike,2023-12-08 19:55:48.207,2023-12-08 20:14:42.829,Grand Army Plaza & Central Park S,6839.10,E 84 St & 1 Ave,7180.02,40.764397,-73.973715,40.775655,-73.950686,member
135,050CBFDF6F040037,electric_bike,2023-12-08 18:28:13.534,2023-12-08 18:33:51.292,Lafayette Ave & Classon Ave,4452.01,Lafayette Ave & Ft Greene Pl,4470.09,40.689004,-73.960239,40.687002,-73.976650,member
604,E4F2B8DB4838348C,electric_bike,2023-12-12 20:49:31.939,2023-12-12 20:59:42.414,1 Ave & E 39 St,6303.01,E 20 St & 2 Ave,5971.08,40.747140,-73.971130,40.735877,-73.982050,member
606,5E63C51D674503FB,classic_bike,2023-12-10 11:46:44.988,2023-12-10 11:49:14.398,1 Ave & E 39 St,6303.01,FDR Drive & E 35 St,6230.04,40.747140,-73.971130,40.744219,-73.971212,member
609,C2C69FE4702C8989,classic_bike,2023-12-09 13:14:59.188,2023-12-09 13:20:44.182,1 Ave & E 39 St,6303.01,FDR Drive & E 35 St,6230.04,40.747140,-73.971130,40.744219,-73.971212,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2204775,BC6B81624DE933B4,electric_bike,2023-12-14 19:24:47.210,2023-12-14 19:27:22.492,E 88 St & Park Ave,7293.1,E 84 St & Park Ave,7243.04,40.781411,-73.955959,40.778627,-73.957721,member
2204776,72F09288D0882EBA,classic_bike,2023-12-23 13:29:32.109,2023-12-23 13:44:25.254,Lafayette Ave & Classon Ave,4452.01,Metropolitan Ave & Bedford Ave,5308.04,40.689004,-73.960239,40.715348,-73.960241,member
2204802,E39FB7340A0192D0,electric_bike,2023-12-19 18:00:15.828,2023-12-19 18:08:07.079,Broadway & W 51 St,6779.04,12 Ave & W 40 St,6765.01,40.762288,-73.983362,40.760875,-74.002777,member
2204834,911D0AFD6897A87E,electric_bike,2023-12-15 17:15:56.355,2023-12-15 17:25:36.311,Broadway & W 51 St,6779.04,12 Ave & W 40 St,6765.01,40.762288,-73.983362,40.760875,-74.002777,casual


In [96]:
x = df.groupby("start_station_id")["start_station_name"].nunique()

In [98]:
df[df["start_station_id"] == "3919.07"]["start_station_name"].unique()

array(['Eastern Pkwy & Franklin Ave',
       'Eastern Pkwy & Franklin Ave (NE Corner)'], dtype=object)

In [99]:
df["test"] = pd.to_numeric(df["start_station_id"], errors="coerce").astype("float32")

In [102]:
stations["sn"] = pd.to_numeric(stations["short_name"], errors="coerce").astype("float32")

In [103]:
stations.dropna(subset=["sn"], inplace=True)

In [108]:
aa = pd.to_numeric(df["start_station_id"], errors="coerce").unique()

In [113]:
aa

array([5578.02, 4042.08, 5569.06, ..., 6028.01, 3877.04, 8772.06],
      shape=(2104,))

In [112]:
stations["sn"].values

array([3680.04, 2009.04, 5074.08, ..., 6915.02, 8381.04, 6932.14],
      shape=(2213,), dtype=float32)

In [111]:
sum = 0
for a in aa:
    if a not in stations["sn"].values:
        print(a)
        sum += 1
print("Total unmatched station IDs:", sum)

5578.02
4042.08
5569.06
5703.13
5779.1
5746.02
4843.01
5414.06
5561.04
6004.07
6989.07
6893.1
5476.03
6617.09
5769.06
4895.03
7235.13
6517.08
7032.06
5679.05
4175.15
6593.14
5297.02
6854.05
7634.01
3865.05
7967.04
6839.1
6958.06
4620.02
3922.02
5721.07
6450.12
4074.14
8072.06
8152.03
6429.01
7304.08
5273.01
6361.03
5419.04
4298.05
4513.06
3620.02
5515.02
6926.01
5805.07
7098.05
6004.06
6422.08
6584.12
5489.03
5779.08
5335.03
6072.11
4190.06
6089.07
4479.06
4452.01
7059.08
5550.05
7144.02
7879.01
4446.03
5820.08
4812.02
6726.01
8319.01
6432.11
6551.11
4904.03
7386.1
7688.12
5779.11
5880.02
5500.02
4470.09
6644.08
5545.01
5561.06
8000.02
5746.14
4010.01
6441.01
8110.04
5433.03
5545.04
3439.03
4225.14
4446.01
7202.07
6319.07
4656.03
8306.03
6676.02
7577.27
8048.04
7028.04
7180.02
5321.08
8196.02
4196.05
3172.01
4107.13
6765.01
5308.04
5422.09
6904.06
6306.01
6190.08
3919.07
5730.08
6847.02
6115.09
4010.13
6331.01
6860.12
5198.04
4336.01
4263.12
5259.06
5114.06
6492.04
5093.01
3379.03
4095